--- Day 12: Rain Risk ---
Your ferry made decent progress toward the island, but the storm came in faster than anyone expected. The ferry needs to take evasive actions!

Unfortunately, the ship's navigation computer seems to be malfunctioning; rather than giving a route directly to safety, it produced extremely circuitous instructions. When the captain uses the PA system to ask if anyone can help, you quickly volunteer.

The navigation instructions (your puzzle input) consists of a sequence of single-character actions paired with integer input values. After staring at them for a few minutes, you work out what they probably mean:

Action N means to move north by the given value.
Action S means to move south by the given value.
Action E means to move east by the given value.
Action W means to move west by the given value.
Action L means to turn left the given number of degrees.
Action R means to turn right the given number of degrees.
Action F means to move forward by the given value in the direction the ship is currently facing.
The ship starts by facing east. Only the L and R actions change the direction the ship is facing. (That is, if the ship is facing east and the next instruction is N10, the ship would move north 10 units, but would still move east if the following action were F.)

For example:

F10
N3
F7
R90
F11
These instructions would be handled as follows:

F10 would move the ship 10 units east (because the ship starts by facing east) to east 10, north 0.
N3 would move the ship 3 units north to east 10, north 3.
F7 would move the ship another 7 units east (because the ship is still facing east) to east 17, north 3.
R90 would cause the ship to turn right by 90 degrees and face south; it remains at east 17, north 3.
F11 would move the ship 11 units south to east 17, south 8.
At the end of these instructions, the ship's Manhattan distance (sum of the absolute values of its east/west position and its north/south position) from its starting position is 17 + 8 = 25.

Figure out where the navigation instructions lead. What is the Manhattan distance between that location and the ship's starting position?

In [13]:
import re

In [117]:
import math

1.0

In [61]:
NORTH,EAST, SOUTH, WEST = 0,1,2,3

In [62]:
ORIENTATION_MAP = {
    NORTH:'N',
    EAST:'E',
    SOUTH:'S',
    WEST:'W'
}


MOVEMENT_MAP = {
    'N':[0,1],
    'E':[1,0],
    'S':[0,-1],
    'W':[-1,0],
    'L':[0,0],
    'R':[0,0],
}


In [63]:
3- 10 % 3

2

In [128]:
class AbstractShip():
    def __init__(self, instructions):
        self.instructions = instructions
        self.coords = [0,0]
        self.orientation = EAST
        self.waypoint_coords = [10,1]
        
    def _move(self,action, value):
        pass
    
    def move_all(self):
        for action, value in self.instructions:
            self._move(action, value)
        return self.coords
    

In [139]:
class Ship(AbstractShip):     
    def _move(self,action, value):
        value_int = int(value)
        if action == 'F':
            direction = ORIENTATION_MAP[self.orientation]
            movements = MOVEMENT_MAP[direction]
        else:
            movements = MOVEMENT_MAP[action]    
        self.coords[0] += (movements[0] * value_int)
        self.coords[1] += (movements[1] * value_int)
        if action == 'L':
            assert value_int % 90 == 0
            self.orientation = (self.orientation - (value_int // 90)) % 4
        if action == 'R':
            assert value_int % 90 == 0
            self.orientation = (self.orientation + (value_int // 90)) % 4
        return self.coords    

In [140]:
test = '''F10
N3
F7
R90
F11'''

In [141]:
instuction_regex = r'(\D)(\d+)'

In [142]:
test_instructions = re.findall(instuction_regex, test)

In [143]:
Ship(test_instructions).move_all()

[17, -8]

In [144]:
input_data = open('data/12.txt','r').read()

In [145]:
instructions = re.findall(instuction_regex, input_data)

In [146]:
res = Ship(instructions).move_all()

In [147]:
abs(res[0]) + abs(res[1])

1221

--- Part Two ---
Before you can give the destination to the captain, you realize that the actual action meanings were printed on the back of the instructions the whole time.

Almost all of the actions indicate how to move a waypoint which is relative to the ship's position:

Action N means to move the waypoint north by the given value.
Action S means to move the waypoint south by the given value.
Action E means to move the waypoint east by the given value.
Action W means to move the waypoint west by the given value.
Action L means to rotate the waypoint around the ship left (counter-clockwise) the given number of degrees.
Action R means to rotate the waypoint around the ship right (clockwise) the given number of degrees.
Action F means to move forward to the waypoint a number of times equal to the given value.
The waypoint starts 10 units east and 1 unit north relative to the ship. The waypoint is relative to the ship; that is, if the ship moves, the waypoint moves with it.

For example, using the same instructions as above:

F10 moves the ship to the waypoint 10 times (a total of 100 units east and 10 units north), leaving the ship at east 100, north 10. The waypoint stays 10 units east and 1 unit north of the ship.
N3 moves the waypoint 3 units north to 10 units east and 4 units north of the ship. The ship remains at east 100, north 10.
F7 moves the ship to the waypoint 7 times (a total of 70 units east and 28 units north), leaving the ship at east 170, north 38. The waypoint stays 10 units east and 4 units north of the ship.
R90 rotates the waypoint around the ship clockwise 90 degrees, moving it to 4 units east and 10 units south of the ship. The ship remains at east 170, north 38.
F11 moves the ship to the waypoint 11 times (a total of 44 units east and 110 units south), leaving the ship at east 214, south 72. The waypoint stays 4 units east and 10 units south of the ship.
After these operations, the ship's Manhattan distance from its starting position is 214 + 72 = 286.

Figure out where the navigation instructions actually lead. What is the Manhattan distance between that location and the ship's starting position?

In [168]:
def rotate(coords, beta):
    '''
    Rotates a vector values x y theta degress
    
    𝑥2=cos𝛽𝑥1−sin𝛽𝑦1
    𝑦2=sin𝛽𝑥1+cos𝛽𝑦1
    '''
    x, y = coords
    beta_r = math.radians(beta)
    x2 = (x*math.cos(beta_r)) - (y*math.sin(beta_r))
    y2 = (x*math.sin(beta_r)) + (y*math.cos(beta_r))
    return [int(x2), int(y2)]

In [169]:
class WaypointShip(AbstractShip):
    '''
    Action N means to move the waypoint north by the given value. 
    Action S means to move the waypoint south by the given value. 
    Action E means to move the waypoint east by the given value. 
    Action W means to move the waypoint west by the given value. 
    Action L means to rotate the waypoint around the ship left (counter-clockwise) the given number of degrees. 
    Action R means to rotate the waypoint around the ship right (clockwise) the given number of degrees. 
    Action F means to move forward to the waypoint a number of times equal to the given value. 
    The waypoint starts 10 units east and 1 unit north relative to the ship. 
    The waypoint is relative to the ship; that is, if the ship moves, the waypoint moves with it.
    '''        
    def _move(self,action, value):
        value_int = int(value)
        if action in ['N','S','E','W']:
            movements = MOVEMENT_MAP[action]
            self.waypoint_coords[0] += (movements[0] * value_int)
            self.waypoint_coords[1] += (movements[1] * value_int)
        elif action == 'L':
            self.waypoint_coords = rotate(self.waypoint_coords, value_int)
        elif action == 'R':
            self.waypoint_coords = rotate(self.waypoint_coords, -value_int)
        elif action == 'F': 
            self.coords[0] += (self.waypoint_coords[0] * value_int)
            self.coords[1] += (self.waypoint_coords[1] * value_int)
        return self.coords

In [170]:
WaypointShip(test_instructions).move_all()

[214, -72]

In [171]:
res = WaypointShip(instructions).move_all()

In [173]:
abs(res[0]) + abs(res[1])

49452